<a href="https://colab.research.google.com/github/daylightzjr/daylightzjr/blob/main/%E6%96%87%E6%9C%AC%E7%9B%B8%E4%BC%BC%E5%BA%A6%E5%AE%9E%E6%88%98follow_step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **文本相似度分析**

##step0 下载与之相关的库(包）

In [22]:
!pip install transformers datasets evaluate accelerate

##Step1 导入相关的包

In [2]:
from transformers import AutoTokenizer,Trainer,TrainingArguments,AutoModelForSequenceClassification
from datasets import load_dataset

##step2 加载数据集

In [23]:
dataset = load_dataset("json",data_files="./train_pair_1w.json",split="train")
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [24]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

##step3 划分数据集

In [25]:
datasets=dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

##step4 数据的预处理

In [26]:
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
token = tokenizer(dataset[0]["sentence1"],dataset[0]["sentence2"],max_length=128,truncation=True)
tokenizer.decode(token.input_ids),token

('[CLS] 找 一 部 小 时 候 的 动 画 片 [SEP] 求 一 部 小 时 候 的 动 画 片 。 谢 了 [SEP]',
 {'input_ids': [101, 2823, 671, 6956, 2207, 3198, 952, 4638, 1220, 4514, 4275, 102, 3724, 671, 6956, 2207, 3198, 952, 4638, 1220, 4514, 4275, 511, 6468, 749, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [27]:
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
def preprocess_function(examples):
    exampled_tokenize = tokenizer(examples["sentence1"],examples["sentence2"],max_length=128,truncation=True)
    exampled_tokenize["labels"] =[float(label) for label in examples["label"]]
    return exampled_tokenize
tokenized_datasets = datasets.map(preprocess_function,batched=True,remove_columns=datasets["train"].column_names)
tokenized_datasets


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [28]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 2617, 2617, 779, 6206, 3221, 3800, 7218, 712, 6572, 2787, 3221, 7444, 6206, 3800, 7218, 1068, 5468, 6572, 2787, 4638, 1450, 102, 1068, 5468, 6572, 2787, 3800, 7218, 4638, 6413, 3221, 7444, 6206, 1044, 3800, 7218, 2094, 6572, 2787, 4638, 749, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0.0}


##step5 创建模型

In [29]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-base",num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##step6 创建评估函数

In [30]:
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [31]:
def eval_metric(eval_pred):
  prediction,labels = eval_pred
  predictions =[int(pre>0.5)for pre in prediction]
  labels = [int(i) for i in labels]
  acc = acc_metric.compute(predictions=predictions,references=labels)
  f1 = f1_metric.compute(predictions=predictions,references=labels)
  acc.update(f1)
  return acc

##step7 创建TrainingArguments

In [32]:
train_args = TrainingArguments(output_dir="./cross_model",      # 输出文件夹
                per_device_train_batch_size=32,  # 训练时的batch_size
                per_device_eval_batch_size=32,   # 验证时的batch_size
                logging_steps=10,                # log 打印的频率
                eval_strategy="epoch",           # 评估策略
                save_strategy="epoch",           # 保存策略
                save_total_limit=3,              # 最大保存数
                learning_rate=2e-5,              # 学习率
                weight_decay=0.01,               # weight_decay
                metric_for_best_model="f1",      # 设定评估指标
                load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp

##step8 创建Trainer

In [33]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model,
          args=train_args,
          tokenizer=tokenizer,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["test"],
          data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
          compute_metrics=eval_metric)

##step9 模型训练

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.110200,0.075774,0.899000,0.870347
2,0.060400,0.071119,0.913500,0.893276
3,0.043800,0.068043,0.917500,0.895371


<ipython-input-31-f42525792fb8>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions =[int(pre>0.5)for pre in prediction]
<ipython-input-31-f42525792fb8>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions =[int(pre>0.5)for pre in prediction]
<ipython-input-31-f42525792fb8>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions =[int(pre>0.5)for pre in prediction]


TrainOutput(global_step=750, training_loss=0.08331266629695892, metrics={'train_runtime': 556.6456, 'train_samples_per_second': 43.115, 'train_steps_per_second': 1.347, 'total_flos': 1554265270970304.0, 'train_loss': 0.08331266629695892, 'epoch': 3.0})

##step10 模型评估

In [35]:
trainer.evaluate(tokenized_datasets["test"])

<ipython-input-31-f42525792fb8>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions =[int(pre>0.5)for pre in prediction]


{'eval_loss': 0.06804341822862625,
 'eval_accuracy': 0.9175,
 'eval_f1': 0.8953709575142677,
 'eval_runtime': 13.8473,
 'eval_samples_per_second': 144.433,
 'eval_steps_per_second': 4.55,
 'epoch': 3.0}

##step11 模型预测

In [37]:
from transformers import pipeline , TextClassificationPipeline

In [38]:
model.config.id2label = {0:"不相似",1:"相似"}



In [39]:
pipe = pipeline("text-classification",model=model,tokenizer=tokenizer,device=0)

In [51]:
result = pipe({"text":"你好","text_pair":"你好吗"},function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': 0.24836276471614838}